In [ ]:
import pandas
import seaborn as sns

sns.set_style("whitegrid")
sns.set_context("poster", font_scale=0.75)
sns.set_palette("colorblind")

alloc_order = ["Table", "ArrayAligned", "ArrayLocked", "ArrayAtomic"]

# Local Benchmarks

## 4K Pages

In [ ]:
pgd = pandas.read_csv("out/filling.csv")
pgd["x"] *= 0.01
pgd = pgd[["x", "alloc", "get_avg", "put_avg"]
          ].melt(id_vars=["x", "alloc"], value_vars=["get_avg", "put_avg"], value_name="time")
g = sns.relplot(data=pgd, kind="line", x="x", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order, hue_order=alloc_order)
# g.set(xticks=[1, 2, 4, 6, 8, 10])
g.set(ylim=(0, 200))
g.set(xlabel="initial filling level")
g.set(ylabel="time in ns")


# Server Benchmarks

In [ ]:
vd = pandas.read_csv("remote/filling_v.csv")
vd["mem"] = "DRAM"
pd = pandas.read_csv("remote/filling_p.csv")
pd["mem"] = "NVRAM"
pgd = pandas.concat([vd, pd], ignore_index=True)

pgd["level"] = pgd["x"] * 0.01
pgd["get"] = pgd["get_avg"]
pgd["put"] = pgd["put_avg"]
pgd = pgd[["level", "alloc", "get", "put", "mem"]
          ].melt(id_vars=["level", "alloc", "mem"], value_vars=["get", "put"], value_name="time")

g = sns.relplot(data=pgd, kind="line", x="level", y="time",
                row="variable", col="mem", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order, hue_order=alloc_order)
g.set_titles("{row_name}, {col_name}")
g.legend.set_title("Allocator")
g.set(xlabel="initial filling level")
g.set(ylabel="time in ns")
g.savefig("../../text/thesis/plot/filling.pdf")
